<a href="https://colab.research.google.com/github/rudycav/TradingStrategy/blob/main/TradingStrategy/%20TradingStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install plotly
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
#!pip install DateTime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#Might need to !pip install these libraries if you don't have them
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_datareader as pdr 
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import figure
from datetime import date
import plotly.io as pio

In [3]:
#Function for getting the stock tickers and start/end timeframe - prepost ignores pre/post market
def training_ticker(ticker="SPY AMD AAPL SNAP TSLA TWTR GME AMC AMZN NFLX", start="2020-01-01", end="2020-12-31"):
  return yf.download(ticker, start=start, end=end, prepost=False)
#test data
def testing_ticker(ticker="SPY AMD AAPL SNAP TSLA TWTR GME AMC AMZN NFLX", start="2021-01-01", end=date.today()):
  return yf.download(ticker, start=start, end=end, prepost=False)

In [4]:
train_df = training_ticker()
test_df = testing_ticker()
#New df contains only these columns for plotly
train_df = train_df[['Close', 'Open', 'High', 'Low']].copy()
test_df = test_df[['Close', 'Open', 'High', 'Low']].copy()
train_df.head()

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed


Close                                                  \
                 AAPL   AMC        AMD       AMZN     GME        NFLX   
Date                                                                    
2020-01-02  75.087502  7.46  49.099998  94.900497  1.5775  329.809998   
2020-01-03  74.357498  7.32  48.599998  93.748497  1.4700  325.899994   
2020-01-06  74.949997  7.12  48.389999  95.143997  1.4625  335.829987   
2020-01-07  74.597504  7.13  48.250000  95.343002  1.3800  330.750000   
2020-01-08  75.797501  6.62  47.830002  94.598503  1.4300  339.260010   

                                                         ...        Low        \
                 SNAP         SPY       TSLA       TWTR  ...       AAPL   AMC   
Date                                                     ...                    
2020-01-02  16.780001  324.869995  28.684000  32.299999  ...  73.797501  7.23   
2020-01-03  16.750000  322.410004  29.534000  31.520000  ...  74.125000  7.31   
2020-01-06  16.780001  323.640015  30.102667  31.639999  ...  73.187500  7.11   
2020-01-07  17.030001  322.730011  31.270666  32.540001  ...  74.370003  7.00   
2020-01-08  16.670000  324.450012  32.809334  33.049999  ...  74.290001  6.61   

                                                                             \
                  AMD       AMZN     GME        NFLX       SNAP         SPY   
Date                                                                          
2020-01-02  46.630001  93.207497  1.5175  324.779999  16.459999  322.529999   
2020-01-03  47.540001  93.224998  1.4600  325.529999  16.350000  321.100006   
2020-01-06  47.860001  93.000000  1.4000  321.200012  16.410000  320.359985   
2020-01-07  48.040001  94.601997  1.3600  330.299988  16.549999  322.239990   
2020-01-08  47.139999  94.321999  1.3525  331.049988  16.650000  322.670013   

                                  
                 TSLA       TWTR  
Date                              
2020-01-02  28.114000  31.959999  
2020-01-03  29.128000  31.260000  
2020-01-06  29.333332  31.160000  
2020-01-07  30.224001  31.719999  
2020-01-08  31.215334  32.349998  

[5 rows x 40 columns]

In [5]:
#Turn floats into integers for more precision in plotting
def smooth_data(data, alpha):
    return data.ewm(alpha=alpha).mean()

train_df = smooth_data(train_df, 0.65)
test_df = smooth_data(test_df, 0.65)

#Relative strength index (RSI) - Overbought/ oversold indicator
def rsi(df, periods = 14, ema = True):
    close_delta = df.Close.diff()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

train_RSI = rsi(train_df)
train_RSI.replace(np.nan,0) #0's are returned until sufficient data is gathered for 14 day period
test_RSI = rsi(test_df)
test_RSI.replace(np.nan,0)

for tickers in train_df.Close:
    #The 3 popular ema strategies are 9, 20, and 50; EMAs are more time sensitive compared to SMAs and reduces lagging
    train_EMA_9 = train_df.ewm(span=9, adjust=False).mean() 
    train_EMA_20 = train_df.ewm(span=20, adjust=False).mean() 
    train_EMA_50 = train_df.ewm(span=50, adjust=False).mean()
    #SMA for comparison; confirms a trend rather than emphasis on recent data; more optimal for golden cross strategy than EMAs
    train_SMA_9 = train_df.rolling(window = 9, min_periods = 1).mean()
    train_SMA_20 = train_df.rolling(window = 20, min_periods = 1).mean()
    train_SMA_50 = train_df.rolling(window = 50, min_periods = 1).mean()

for tickers in test_df.Close:
    test_EMA_9 = test_df.ewm(span=9, adjust=False).mean() 
    test_EMA_20 = test_df.ewm(span=20, adjust=False).mean() 
    test_EMA_50 = test_df.ewm(span=50, adjust=False).mean()

    test_SMA_9 = test_df.rolling(window = 9, min_periods = 1).mean()
    test_SMA_20 = test_df.rolling(window = 20, min_periods = 1).mean()
    test_SMA_50 = test_df.rolling(window = 50, min_periods = 1).mean()


stock_tickers = ['SPY', 'AMD', 'AAPL', 'SNAP', 'TSLA', 'TWTR', 'GME', 'AMC', 'AMZN', 'NFLX']
#Flag for buy/sell signal - 1.0 when 9SMA > 20SMA & 50SMA; 0.0 when 9SMA < 20SMA & 50SMA
def buy_sell_signal(df, MA9_df, MA20_df, MA50_df, ticker):
  for price in df.values:
      golden_cross = np.where((MA9_df > MA20_df) & (MA9_df > MA50_df), 1.0, 0.0)
    
  df['Flag Signal', ticker] = golden_cross
  # Signal becomes 1 when 0 changes to 1 - golden cross; Buy, -1 when 1 changes to 0 - short EMA has declined below long EMAs; Sell
  df['Buy/Sell', ticker] = df['Flag Signal', ticker].diff() 
  return df['Buy/Sell', ticker]


#ROI percentage tracker for whenever a Sell occurs; Total gains 
def roi_tracker(df, SMA9_df, EMA9_df, ticker):
  Ssell_array = SMA9_df[('Close', ticker)][df['Buy/Sell', ticker] == -1].to_numpy()
  Sbuy_array = SMA9_df[('Close', ticker)][df['Buy/Sell', ticker] == 1].to_numpy()
  Esell_array = EMA9_df[('Close', ticker)][df['Buy/Sell', ticker] == -1].to_numpy()
  Ebuy_array = EMA9_df[('Close', ticker)][df['Buy/Sell', ticker] == 1].to_numpy()
  #Subtracting Sell price from previous Buy price 
  while len(Sbuy_array) > len(Ssell_array) or len(Ebuy_array) > len(Esell_array):
    #If latest Buy signal triggered and no Sell signal yet, removes the last element from Buy array
    SROI = Ssell_array - Sbuy_array[:-1].copy()
    EROI = Esell_array - Ebuy_array[:-1].copy()
    break
  else:
    SROI = Ssell_array - Sbuy_array
    EROI = Esell_array - Ebuy_array
  print('SMA Performance')
  print('----------------')

  for index, value in enumerate(SROI):
    print(SMA9_df[('Close', ticker)][df['Buy/Sell', ticker] == -1].index[index].date(), ':', round(value, 2),'%')
  print('---------------------')
  print('SMA Performance Total: ', round(sum(SROI), 2), '%')
  print()
  print('EMA Performance')
  print('----------------')

  for index, value in enumerate(EROI):
    print(EMA9_df[('Close', ticker)][df['Buy/Sell', ticker] == -1].index[index].date(), ':', round(value, 2),'%')
  print('----------------------')
  print('EMA Performance Total: ', round(sum(EROI), 2), '%')


#Plot to show the indicators and daily close from 2021-01-01 - today
def technical_analysis_plot(df, MA9_df, MA20_df, MA50_df, RSI_df, ticker, opacity=0.7):
  fig = go.Figure(data=[go.Candlestick(x=df.index,
                  open=df[('Open', ticker)],
                  high=df[('High', ticker)],
                  low=df[('Low', ticker)],
                  close=df[('Close', ticker)],
                  showlegend=False)])
  #9 SMA              
  fig.add_trace(go.Scatter(x=df.index, 
                          y=MA9_df[('Close', ticker)], 
                          opacity=opacity, 
                          line=dict(color='orange', width=2), 
                          name='9 SMA'))
  #20 SMA
  fig.add_trace(go.Scatter(x=df[('Close', ticker)].index, 
                          y=MA20_df[('Close', ticker)], 
                          opacity=opacity, 
                          line=dict(color='blue', width=2), 
                          name='20 SMA'))
  #50 SMA
  fig.add_trace(go.Scatter(x=df[('Close', ticker)].index, 
                          y=MA50_df[('Close', ticker)], 
                          opacity=opacity, 
                          line=dict(color='purple', width=2), 
                          name='50 SMA'))
  #Buy indicator
  fig.add_trace(go.Scatter(x=df[df['Buy/Sell', ticker]==1].index, 
                          y=MA9_df[('Close', ticker)][df['Buy/Sell', ticker] == 1],       
                            name='Buy',  mode = 'markers', opacity=1,
                            marker =dict(size=14,color='#16FF32',symbol='triangle-up'),
                            hovertemplate=('BUY on %{x}')))
  #Sell indicator
  fig.add_trace(go.Scatter(x=df[df['Buy/Sell', ticker]==-1].index, 
                          y=MA9_df[('Close', ticker)][df['Buy/Sell', ticker] == -1],       
                            name='Sell',  mode = 'markers', opacity=1,
                            marker =dict(size=14,color='black',symbol='triangle-down'),
                            hovertemplate=('SELL on %{x}')))
  #RSI
  fig.add_trace(go.Scatter(x=RSI_df.index, y=RSI_df[ticker],
                      mode='lines',
                      name='RSI',
                      line=dict(color="Black", width=1), yaxis="y2"))

  fig.update_layout(
      autosize=False,
      width=1750,
      height=600,
      title=ticker, title_x=0.5,
      yaxis_title='Price',
      yaxis_domain=[0.3, 1],
      yaxis2={"domain": [0, 0.20]},
      xaxis_rangeslider_visible=False)

  fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])]) #Excudes empty data on weekends
  fig.show()

In [6]:
#SMA
SMA_train_SPY = buy_sell_signal(train_df, train_SMA_9.Close.SPY.values, train_SMA_20.Close.SPY.values, train_SMA_50.Close.SPY.values, stock_tickers[0])
SMA_test_SPY = buy_sell_signal(test_df, test_SMA_9.Close.SPY.values, test_SMA_20.Close.SPY.values, test_SMA_50.Close.SPY.values, stock_tickers[0])

SMA_train_AMD  = buy_sell_signal(train_df, train_SMA_9.Close.AMD.values, train_SMA_20.Close.AMD.values, train_SMA_50.Close.AMD .values, stock_tickers[1])
SMA_test_AMD  = buy_sell_signal(test_df, test_SMA_9.Close.AMD.values, test_SMA_20.Close.AMD.values, test_SMA_50.Close.AMD.values, stock_tickers[1])

SMA_train_AAPL  = buy_sell_signal(train_df, train_SMA_9.Close.AAPL.values, train_SMA_20.Close.AAPL.values, train_SMA_50.Close.AAPL.values, stock_tickers[2])
SMA_test_AAPL  = buy_sell_signal(test_df, test_SMA_9.Close.AAPL.values, test_SMA_20.Close.AAPL.values, test_SMA_50.Close.AAPL.values, stock_tickers[2])

SMA_train_SNAP  = buy_sell_signal(train_df, train_SMA_9.Close.SNAP.values, train_SMA_20.Close.SNAP.values, train_SMA_50.Close.SNAP.values, stock_tickers[3])
SMA_test_SNAP  = buy_sell_signal(test_df, test_SMA_9.Close.SNAP.values, test_SMA_20.Close.SNAP.values, test_SMA_50.Close.SNAP.values, stock_tickers[3])

SMA_train_TSLA  = buy_sell_signal(train_df, train_SMA_9.Close.TSLA.values, train_SMA_20.Close.TSLA.values, train_SMA_50.Close.TSLA.values, stock_tickers[4])
SMA_test_TSLA  = buy_sell_signal(test_df, test_SMA_9.Close.TSLA.values, test_SMA_20.Close.TSLA.values, test_SMA_50.Close.TSLA.values, stock_tickers[4])

SMA_train_TWTR  = buy_sell_signal(train_df, train_SMA_9.Close.TWTR.values, train_SMA_20.Close.TWTR.values, train_SMA_50.Close.TWTR.values, stock_tickers[5])
SMA_test_TWTR  = buy_sell_signal(test_df, test_SMA_9.Close.TWTR.values, test_SMA_20.Close.TWTR.values, test_SMA_50.Close.TWTR.values, stock_tickers[5])

SMA_train_GME  = buy_sell_signal(train_df, train_SMA_9.Close.GME.values, train_SMA_20.Close.GME.values, train_SMA_50.Close.GME.values, stock_tickers[6])
SMA_test_GME  = buy_sell_signal(test_df, test_SMA_9.Close.GME.values, test_SMA_20.Close.GME.values, test_SMA_50.Close.GME.values, stock_tickers[6])

SMA_train_AMC  = buy_sell_signal(train_df, train_SMA_9.Close.AMC.values, train_SMA_20.Close.AMC.values, train_SMA_50.Close.AMC.values, stock_tickers[7])
SMA_test_AMC  = buy_sell_signal(test_df, test_SMA_9.Close.AMC.values, test_SMA_20.Close.AMC.values, test_SMA_50.Close.AMC.values, stock_tickers[7])

SMA_train_AMZN  = buy_sell_signal(train_df, train_SMA_9.Close.AMZN.values, train_SMA_20.Close.AMZN.values, train_SMA_50.Close.AMZN.values, stock_tickers[8])
SMA_test_AMZN  = buy_sell_signal(test_df, test_SMA_9.Close.AMZN.values, test_SMA_20.Close.AMZN.values, test_SMA_50.Close.AMZN.values, stock_tickers[8])

SMA_train_NFLX  = buy_sell_signal(train_df, train_SMA_9.Close.NFLX.values, train_SMA_20.Close.NFLX.values, train_SMA_50.Close.NFLX.values, stock_tickers[9])
SMA_test_NFLX  = buy_sell_signal(test_df, test_SMA_9.Close.NFLX.values, test_SMA_20.Close.NFLX.values, test_SMA_50.Close.NFLX.values, stock_tickers[9])

In [7]:
train_df.head()

Close                                                        \
                 AAPL       AMC        AMD       AMZN       GME        NFLX   
Date                                                                          
2020-01-02  75.087502  7.460000  49.099998  94.900497  1.577500  329.809998   
2020-01-03  74.546758  7.356296  48.729628  94.047164  1.497870  326.913699   
2020-01-06  74.820605  7.195823  48.498980  94.792042  1.473850  332.968903   
2020-01-07  74.673380  7.152386  48.334678  95.155622  1.411918  331.504643   
2020-01-08  75.407916  6.804508  48.004906  94.791583  1.423733  336.572247   

                                                         ... Flag Signal  \
                 SNAP         SPY       TSLA       TWTR  ...        TWTR   
Date                                                     ...               
2020-01-02  16.780001  324.869995  28.684000  32.299999  ...         0.0   
2020-01-03  16.757778  323.047779  29.313630  31.722222  ...         0.0   
2020-01-06  16.772870  323.449976  29.849478  31.666383  ...         0.0   
2020-01-07  16.942551  322.974869  30.787324  32.242886  ...         0.0   
2020-01-08  16.764458  323.938775  32.108570  32.770280  ...         0.0   

           Buy/Sell Flag Signal Buy/Sell Flag Signal Buy/Sell Flag Signal  \
               TWTR         GME      GME         AMC      AMC        AMZN   
Date                                                                        
2020-01-02      NaN         0.0      NaN         0.0      NaN         0.0   
2020-01-03      0.0         0.0      0.0         0.0      0.0         0.0   
2020-01-06      0.0         0.0      0.0         0.0      0.0         0.0   
2020-01-07      0.0         0.0      0.0         0.0      0.0         0.0   
2020-01-08      0.0         0.0      0.0         0.0      0.0         0.0   

           Buy/Sell Flag Signal Buy/Sell  
               AMZN        NFLX     NFLX  
Date                                      
2020-01-02      NaN         0.0      NaN  
2020-01-03      0.0         0.0      0.0  
2020-01-06      0.0         0.0      0.0  
2020-01-07      0.0         0.0      0.0  
2020-01-08      0.0         0.0      0.0  

[5 rows x 60 columns]

In [8]:
#AAPL train
roi_tracker(train_df, train_SMA_9, train_EMA_9, stock_tickers[2])
technical_analysis_plot(train_df, train_SMA_9, train_SMA_20, train_SMA_50, train_RSI, stock_tickers[2])
#AAPL test
roi_tracker(test_df, test_SMA_9, test_EMA_9, stock_tickers[2])
technical_analysis_plot(test_df, test_SMA_9, test_SMA_20, test_SMA_50, test_RSI, stock_tickers[2])


SMA Performance
----------------
2020-02-25 : 2.79 %
2020-09-14 : 51.23 %
2020-10-27 : 0.73 %
2020-12-02 : 1.36 %
---------------------
SMA Performance Total:  56.11 %

EMA Performance
----------------
2020-02-25 : 1.21 %
2020-09-14 : 49.56 %
2020-10-27 : -0.42 %
2020-12-02 : 1.65 %
----------------------
EMA Performance Total:  52.0 %


SMA Performance
----------------
2021-01-19 : -0.3 %
2021-02-16 : 6.04 %
2021-05-05 : 5.04 %
2021-08-06 : 17.21 %
2021-09-20 : 3.37 %
2022-01-13 : 28.14 %
2022-02-16 : -0.46 %
2022-04-14 : 4.35 %
2022-08-30 : 22.48 %
2022-12-07 : -1.19 %
---------------------
SMA Performance Total:  84.69 %

EMA Performance
----------------
2021-01-19 : -0.3 %
2021-02-16 : 6.05 %
2021-05-05 : 3.09 %
2021-08-06 : 16.95 %
2021-09-20 : 2.23 %
2022-01-13 : 27.33 %
2022-02-16 : -0.61 %
2022-04-14 : 1.56 %
2022-08-30 : 20.75 %
2022-12-07 : -2.21 %
----------------------
EMA Performance Total:  74.85 %


In [9]:
#AMC train
roi_tracker(train_df, train_SMA_9, train_EMA_9, stock_tickers[7])
technical_analysis_plot(train_df, train_EMA_9, train_EMA_20, train_EMA_50, train_RSI, stock_tickers[7])
#AMC test
roi_tracker(test_df, test_SMA_9, test_EMA_9, stock_tickers[7])
technical_analysis_plot(test_df, test_SMA_9, test_SMA_20, test_SMA_50, test_RSI, stock_tickers[7])


SMA Performance
----------------
2020-01-29 : -0.03 %
2020-03-02 : -0.13 %
2020-06-23 : 1.65 %
2020-09-18 : 1.21 %
2020-12-15 : -0.05 %
---------------------
SMA Performance Total:  2.64 %

EMA Performance
----------------
2020-01-29 : -0.06 %
2020-03-02 : -0.41 %
2020-06-23 : 1.46 %
2020-09-18 : 1.0 %
2020-12-15 : -0.23 %
----------------------
EMA Performance Total:  1.77 %


SMA Performance
----------------
2021-02-16 : 4.46 %
2021-04-01 : 3.19 %
2021-05-07 : 0.03 %
2021-07-07 : 43.75 %
2021-09-23 : 2.33 %
2021-11-30 : -1.22 %
2022-04-13 : 2.23 %
2022-08-03 : 2.06 %
2022-08-24 : 2.63 %
2022-12-09 : -0.07 %
---------------------
SMA Performance Total:  59.39 %

EMA Performance
----------------
2021-02-16 : 4.31 %
2021-04-01 : 2.73 %
2021-05-07 : -0.52 %
2021-07-07 : 41.8 %
2021-09-23 : 0.86 %
2021-11-30 : -1.57 %
2022-04-13 : 0.43 %
2022-08-03 : 2.51 %
2022-08-24 : -0.66 %
2022-12-09 : -0.29 %
----------------------
EMA Performance Total:  49.61 %


In [10]:
#AMD train
roi_tracker(train_df, train_SMA_9, train_EMA_9, stock_tickers[1])
technical_analysis_plot(train_df, train_SMA_9, train_SMA_20, train_SMA_50, train_RSI, stock_tickers[1])
#AMD test
roi_tracker(test_df, test_SMA_9, test_EMA_9, stock_tickers[1])
technical_analysis_plot(test_df, test_SMA_9, test_SMA_20, test_SMA_50, test_RSI, stock_tickers[1])



SMA Performance
----------------
2020-02-05 : 0.1 %
2020-03-02 : 0.55 %
2020-05-07 : 4.89 %
2020-06-03 : -0.57 %
2020-06-25 : -0.16 %
2020-09-14 : 29.12 %
2020-10-22 : 0.55 %
2020-12-30 : 12.27 %
---------------------
SMA Performance Total:  46.75 %

EMA Performance
----------------
2020-02-05 : -0.34 %
2020-03-02 : -1.91 %
2020-05-07 : 3.28 %
2020-06-03 : -0.87 %
2020-06-25 : -0.52 %
2020-09-14 : 27.14 %
2020-10-22 : -0.71 %
2020-12-30 : 11.58 %
----------------------
EMA Performance Total:  37.65 %


SMA Performance
----------------
2021-01-21 : -0.94 %
2021-02-22 : -0.38 %
2021-05-10 : -1.67 %
2021-07-20 : 9.48 %
2021-08-24 : 15.28 %
2021-09-15 : -1.37 %
2021-12-09 : 41.3 %
2022-01-10 : -2.16 %
2022-04-06 : -2.26 %
2022-06-16 : -3.13 %
2022-08-24 : 7.73 %
2022-12-09 : 6.0 %
---------------------
SMA Performance Total:  67.9 %

EMA Performance
----------------
2021-01-21 : -1.35 %
2021-02-22 : -0.73 %
2021-05-10 : -2.8 %
2021-07-20 : 8.67 %
2021-08-24 : 13.57 %
2021-09-15 : -2.38 %
2021-12-09 : 37.81 %
2022-01-10 : -6.47 %
2022-04-06 : -5.47 %
2022-06-16 : -7.62 %
2022-08-24 : 6.55 %
2022-12-09 : 3.57 %
----------------------
EMA Performance Total:  43.34 %


In [11]:
#AMZN train
roi_tracker(train_df, train_SMA_9, train_EMA_9, stock_tickers[8])
technical_analysis_plot(train_df, train_SMA_9, train_EMA_20, train_SMA_50, train_RSI, stock_tickers[8])
#AMZN test
roi_tracker(test_df, test_SMA_9, test_EMA_9, stock_tickers[8])
technical_analysis_plot(test_df, test_SMA_9, test_SMA_20, test_SMA_50, test_RSI, stock_tickers[8])


SMA Performance
----------------
2020-02-28 : 7.97 %
2020-05-12 : 19.88 %
2020-07-31 : 34.08 %
2020-09-14 : 9.96 %
2020-10-28 : -0.3 %
---------------------
SMA Performance Total:  71.6 %

EMA Performance
----------------
2020-02-28 : 5.18 %
2020-05-12 : 19.5 %
2020-07-31 : 33.72 %
2020-09-14 : 6.11 %
2020-10-28 : -1.38 %
----------------------
EMA Performance Total:  63.13 %


SMA Performance
----------------
2021-02-19 : 5.2 %
2021-05-10 : 8.98 %
2021-07-28 : 15.67 %
2021-09-23 : -1.96 %
2021-11-02 : 0.58 %
2021-12-03 : 6.66 %
2022-04-12 : 6.07 %
2022-08-26 : 25.38 %
---------------------
SMA Performance Total:  66.57 %

EMA Performance
----------------
2021-02-19 : 3.66 %
2021-05-10 : 5.56 %
2021-07-28 : 15.35 %
2021-09-23 : -1.94 %
2021-11-02 : -0.45 %
2021-12-03 : 4.88 %
2022-04-12 : 1.87 %
2022-08-26 : 24.69 %
----------------------
EMA Performance Total:  53.62 %


In [12]:
#GME train
roi_tracker(train_df, train_SMA_9, train_EMA_9, stock_tickers[6])
technical_analysis_plot(train_df, train_SMA_9, train_SMA_20, train_SMA_50, train_RSI, stock_tickers[6])
#GME test
roi_tracker(test_df, test_SMA_9, test_EMA_9, stock_tickers[6])
technical_analysis_plot(test_df, test_SMA_9, test_SMA_20, test_SMA_50, test_RSI, stock_tickers[6])



SMA Performance
----------------
2020-03-30 : -0.02 %
2020-05-13 : 0.27 %
2020-06-19 : 0.0 %
2020-11-04 : 1.97 %
2020-12-17 : 0.66 %
---------------------
SMA Performance Total:  2.89 %

EMA Performance
----------------
2020-03-30 : 0.01 %
2020-05-13 : 0.13 %
2020-06-19 : 0.01 %
2020-11-04 : 1.87 %
2020-12-17 : 0.54 %
----------------------
EMA Performance Total:  2.55 %


SMA Performance
----------------
2021-02-11 : 18.23 %
2021-03-29 : 25.57 %
2021-06-21 : 15.31 %
2021-09-20 : 4.5 %
2021-12-02 : 3.42 %
2022-04-19 : 7.96 %
2022-07-05 : -0.23 %
2022-08-03 : 1.42 %
2022-08-25 : 0.7 %
2022-11-10 : -0.56 %
2022-12-06 : -0.59 %
---------------------
SMA Performance Total:  75.73 %

EMA Performance
----------------
2021-02-11 : 14.52 %
2021-03-29 : 22.16 %
2021-06-21 : 13.05 %
2021-09-20 : 3.09 %
2021-12-02 : 0.14 %
2022-04-19 : 5.07 %
2022-07-05 : -0.51 %
2022-08-03 : 1.27 %
2022-08-25 : -1.68 %
2022-11-10 : -1.11 %
2022-12-06 : -0.56 %
----------------------
EMA Performance Total:  55.44 %


In [13]:
#NFLX train
roi_tracker(train_df, train_SMA_9, train_EMA_9, stock_tickers[9])
technical_analysis_plot(train_df, train_SMA_9, train_SMA_20, train_SMA_50, train_RSI, stock_tickers[9])
#NFLX test
roi_tracker(test_df, test_SMA_9, test_EMA_9, stock_tickers[9])
technical_analysis_plot(test_df, test_SMA_9, test_SMA_20, test_SMA_50, test_RSI, stock_tickers[9])


SMA Performance
----------------
2020-03-04 : 39.74 %
2020-06-01 : 70.35 %
2020-07-28 : 68.49 %
2020-08-17 : -3.98 %
2020-09-15 : 14.96 %
2020-10-26 : 19.32 %
---------------------
SMA Performance Total:  208.88 %

EMA Performance
----------------
2020-03-04 : 40.06 %
2020-06-01 : 67.08 %
2020-07-28 : 64.01 %
2020-08-17 : -1.77 %
2020-09-15 : 2.76 %
2020-10-26 : 6.18 %
----------------------
EMA Performance Total:  178.32 %


SMA Performance
----------------
2021-02-26 : 31.11 %
2021-04-26 : -1.03 %
2021-07-23 : 27.22 %
2021-09-27 : 64.36 %
2021-11-16 : 71.56 %
2021-12-01 : -10.39 %
2022-08-29 : 47.26 %
2022-10-12 : 3.88 %
2022-11-11 : 33.59 %
---------------------
SMA Performance Total:  267.55 %

EMA Performance
----------------
2021-02-26 : 20.12 %
2021-04-26 : -11.64 %
2021-07-23 : 19.03 %
2021-09-27 : 62.06 %
2021-11-16 : 73.19 %
2021-12-01 : -19.91 %
2022-08-29 : 41.32 %
2022-10-12 : -0.51 %
2022-11-11 : 27.51 %
----------------------
EMA Performance Total:  211.18 %


In [14]:
#SPY train
roi_tracker(train_df, train_SMA_9, train_EMA_9, stock_tickers[0])
technical_analysis_plot(train_df, train_SMA_9, train_SMA_20, train_SMA_50, train_RSI, stock_tickers[0])
#SPY test
roi_tracker(test_df, test_SMA_9, test_EMA_9, stock_tickers[0])
technical_analysis_plot(test_df, test_SMA_9, test_SMA_20, test_SMA_50, test_RSI, stock_tickers[0])



SMA Performance
----------------
2020-02-04 : 1.67 %
2020-02-27 : -2.42 %
2020-06-23 : 28.88 %
2020-09-15 : 31.0 %
2020-10-28 : 4.69 %
---------------------
SMA Performance Total:  63.81 %

EMA Performance
----------------
2020-02-04 : 0.53 %
2020-02-27 : -8.81 %
2020-06-23 : 31.34 %
2020-09-15 : 27.75 %
2020-10-28 : 2.23 %
----------------------
EMA Performance Total:  53.05 %


SMA Performance
----------------
2021-02-04 : 2.5 %
2021-03-02 : 5.94 %
2021-05-13 : 27.88 %
2021-09-17 : 31.42 %
2021-12-01 : 19.63 %
2022-01-14 : 4.54 %
2022-04-13 : 6.76 %
2022-08-30 : 22.71 %
---------------------
SMA Performance Total:  121.39 %

EMA Performance
----------------
2021-02-04 : 3.21 %
2021-03-02 : 2.9 %
2021-05-13 : 24.12 %
2021-09-17 : 29.72 %
2021-12-01 : 15.61 %
2022-01-14 : 3.0 %
2022-04-13 : 3.58 %
2022-08-30 : 19.33 %
----------------------
EMA Performance Total:  101.48 %


In [15]:
#TSLA train
roi_tracker(train_df, train_SMA_9, train_EMA_9, stock_tickers[4])
technical_analysis_plot(train_df, train_SMA_9, train_SMA_20, train_SMA_50, train_RSI, stock_tickers[4])
#TSLA test
roi_tracker(test_df, test_SMA_9, test_EMA_9, stock_tickers[4])
technical_analysis_plot(test_df, test_SMA_9, test_SMA_20, test_SMA_50, test_RSI, stock_tickers[4])


SMA Performance
----------------
2020-03-03 : 20.6 %
2020-08-04 : 55.71 %
2020-09-14 : 33.42 %
2020-10-27 : 2.4 %
---------------------
SMA Performance Total:  112.12 %

EMA Performance
----------------
2020-03-03 : 18.23 %
2020-08-04 : 54.48 %
2020-09-14 : 27.48 %
2020-10-27 : 4.32 %
----------------------
EMA Performance Total:  104.51 %


SMA Performance
----------------
2021-02-09 : 11.08 %
2021-05-04 : -1.51 %
2021-07-19 : 3.43 %
2021-08-24 : 11.22 %
2021-11-18 : 121.71 %
2021-12-08 : -12.4 %
2022-01-19 : -8.17 %
2022-04-18 : 37.95 %
2022-08-29 : 56.5 %
2022-09-30 : -6.95 %
---------------------
SMA Performance Total:  212.87 %

EMA Performance
----------------
2021-02-09 : 11.78 %
2021-05-04 : -4.43 %
2021-07-19 : 0.93 %
2021-08-24 : 8.77 %
2021-11-18 : 120.51 %
2021-12-08 : -14.12 %
2022-01-19 : -10.71 %
2022-04-18 : 27.17 %
2022-08-29 : 55.11 %
2022-09-30 : -13.04 %
----------------------
EMA Performance Total:  181.95 %


In [16]:
#TWTR train
roi_tracker(train_df, train_SMA_9, train_EMA_9, stock_tickers[5])
technical_analysis_plot(train_df, train_SMA_9, train_SMA_20, train_SMA_50, train_RSI, stock_tickers[5])
#TWTR test
roi_tracker(test_df, test_SMA_9, test_EMA_9, stock_tickers[5])
technical_analysis_plot(test_df, test_SMA_9, test_SMA_20, test_SMA_50, test_RSI, stock_tickers[5])



SMA Performance
----------------
2020-02-05 : 0.71 %
2020-03-03 : 2.14 %
2020-06-24 : 4.93 %
2020-09-16 : 6.35 %
2020-11-04 : 6.15 %
---------------------
SMA Performance Total:  20.28 %

EMA Performance
----------------
2020-02-05 : 0.63 %
2020-03-03 : 1.29 %
2020-06-24 : 4.97 %
2020-09-16 : 5.83 %
2020-11-04 : 3.83 %
----------------------
EMA Performance Total:  16.55 %


SMA Performance
----------------
2021-03-10 : 19.99 %
2021-04-26 : 1.56 %
2021-07-26 : 8.58 %
2021-08-10 : -0.32 %
2021-10-28 : -1.06 %
2022-05-13 : 11.85 %
2022-08-29 : 3.52 %
2022-11-25 : 12.23 %
2022-11-29 : -0.0 %
2022-12-01 : -0.0 %
2022-12-05 : -0.0 %
2022-12-07 : -0.0 %
2022-12-09 : -0.0 %
---------------------
SMA Performance Total:  56.34 %

EMA Performance
----------------
2021-03-10 : 17.42 %
2021-04-26 : -0.31 %
2021-07-26 : 9.02 %
2021-08-10 : -1.11 %
2021-10-28 : -2.98 %
2022-05-13 : 10.35 %
2022-08-29 : 2.88 %
2022-11-25 : 12.06 %
2022-11-29 : 0.0 %
2022-12-01 : 0.0 %
2022-12-05 : 0.0 %
2022-12-07 : 0.0 %
2022-12-09 : 0.0 %
----------------------
EMA Performance Total:  47.34 %
